In [ ]:
#  This is filename:  uhlig_simul_rand.ipynb

#  The purpose of this notebook is to compare
#  the model's performance to data, by simulating
#  the model using the following input:  a set of 
#  randomly generated values for the exogenous 
#  stochastic processes.  The statistical properties
#  of the simulated series can then be compared
#  to the statistical properties of data.


#   The inputs into this notebook are the matices 
#   PP,  QQ,  RR,  SS  and  WW
#   as well as the matrices NN and Sigma_EPS,
#   which represent the exogenous stochastic
#   processes

#   The output of this notebook are the matrices  
#   Response and Response_named, which
#   contain simulation results


import numpy as np
import numpy.matlib as npm
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as plt
import pylab

import pandas as pd

import statsmodels.api as sm

from IPython.display import display, HTML

np.set_printoptions(precision=3, suppress=False)
pd.set_option('display.float_format', '{:.4e}'.format)



##  Some preliminaries

#   For this particular simulation, I set initial values for 
#   to zero.  If you have other initial assumptions,
#   you can insert a line here

X_sv_count = m_states
X_sv = np.zeros(X_sv_count)
Resp_Initial = np.matrix(X_sv.reshape(X_sv_count, 1))


##  USER INPUT:  Choose how long the simulation
##  is going to last.  And how much of the initial results
##  to discard

sim_LENGTH = 400
sim_LENGTH_discard = 100
sim_LENGTH_total = sim_LENGTH + sim_LENGTH_discard 


#  Generate the matrices  given_EPS and given_ZV  using
#  a random number generator, NN and Sigma_EPS

given_EPS = np.zeros((k_exog, sim_LENGTH))

given_EPS =  np.random.normal(npm.zeros((k_exog, sim_LENGTH_total)), npm.ones((k_exog, sim_LENGTH_total)))

Sigma_EPS = np.matrix(Sigma_EPS)  #  Sigma_EPS is squared, i.e. var-covariances

Sig_fact = np.linalg.cholesky(Sigma_EPS)  # This is lower triangular

given_EPS =  dot(Sig_fact, given_EPS)

given_ZV = np.zeros(k_exog).reshape(k_exog,1)

#   Note the first column of the matrix of exogenous shocks is zero
                    
ZV_counter = 0

for ZV_counter in range(0, sim_LENGTH_total):
        given_ZV_col = dot(NN,given_ZV[:,ZV_counter]) + given_EPS[:,ZV_counter]
        given_ZV = np.hstack((given_ZV,given_ZV_col))

#   Let's drop the first column of zeros        

col_counter_ZV = given_ZV.shape[1]
given_ZV = np.matrix(given_ZV[:,1:col_counter_ZV])                    

                    

##  Now to calculate the simulation


#  Initializing the Response matrix

Response =  np.zeros((m_states+n_endog)).reshape(m_states+n_endog,1)
Response = np.matrix(Response)


#  Assembling first column of  the Response matrix
#  which are the initial values, here all zeros

Response_0  = np.vstack((Resp_Initial.reshape(m_states,1), np.zeros(n_endog).reshape(n_endog,1)))
Response = Response_0 


# Computing second column of the Response matrix

RespLag_state_1 = dot(PP,Resp_Initial)
RespLag_jump_1 = dot(RR,Resp_Initial)
RespContemp_state_1 = dot(QQ,given_ZV[:,0])
RespContemp_jump_1 = dot(SS,given_ZV[:,0])

Response_1 = np.zeros(m_states+n_endog).reshape((m_states+n_endog),1)
Resp_state_1 = RespLag_state_1 + RespContemp_state_1
Resp_jump_1 = RespLag_jump_1 + RespContemp_jump_1
Response_1 = np.vstack((Resp_state_1,Resp_jump_1))

Response = np.hstack((Response, Response_1 ))

# Computing the remaining columns of the Response matrix

sim_LENGTH_counter = 0

for sim_LENGTH_counter in range(1, sim_LENGTH_total):

        Response_col = np.zeros(m_states+n_endog).reshape((m_states+n_endog),1)
        RespLag_state_col = np.zeros(m_states).reshape(m_states,1)
        RespLag_jump_col = np.zeros(n_endog).reshape(n_endog,1)
        RespContemp_state_col = np.zeros(m_states).reshape(m_states,1)
        RespContemp_jump_col = np.zeros(n_endog).reshape(n_endog,1)     
        
        RespLag_state_col = dot(PP,Response[0:(m_states), sim_LENGTH_counter])
        RespLag_jump_col = dot(RR,Response[0:(m_states), sim_LENGTH_counter])
        RespContemp_state_col = dot(QQ,given_ZV[:,sim_LENGTH_counter])
        RespContemp_jump_col = dot(SS,given_ZV[:,sim_LENGTH_counter])

        Resp_state_col = RespLag_state_col + RespContemp_state_col
        Resp_jump_col = RespLag_jump_col + RespContemp_jump_col
        Response_col = np.vstack((Resp_state_col,Resp_jump_col))

        Response = np.hstack((Response, Response_col))

#  Drop column of initial values, which are all zeros

col_counter_Resp = Response.shape[1]
Response = Response[:,1:col_counter_Resp]

#   Discard the first  'sim_LENGTH_discard'  simulated values

col_counter_Resp = Response.shape[1]
Response = Response[:, sim_LENGTH_discard:col_counter_Resp]

col_counter_ZV = given_ZV.shape[1]
given_ZV = given_ZV[:, sim_LENGTH_discard:col_counter_Resp]
        
#print('Response = ')
#print(Response)
#print('   ')
#print('Response shape = ', Response.shape)                 
#print('   ')


temp_exog_var = np.hstack((np.zeros(k_exog).reshape(k_exog,1), given_ZV))

Response = np.vstack((Response, given_ZV))

Response_named = np.hstack((Varnames, Response))       

#print('Response_named = ')
#print(Response_named)
#print('   ')
#print('Response_named shape = ', Response_named.shape)                 
#print('   ')

# For convenience with some other notebooks

Resp_mat = Response
Resp_mat_named = Response_named


print('  ')    
print('Done. You have the matrices "Response" and "Response_named" ')
print('which contain the models simulated reponses to')
print('given values for the exogenous stochastic processes.')
#print('If you want, you can download these into an excel file')
#print('or use them to make plots.')
print('  ')   
print('  ')   
print('Here are the UNFILTERED SIMULATED series.')
print('Total Simulation Length = ', sim_LENGTH_total)
print('Number of Initial Values Discarded = ', sim_LENGTH_discard)
#print('  ')   
#print('Response_named = ')
#print(Response_named)
#print('  ')   
